In [1]:
# import gym
# 
# env = gym.make("MountainCar-v0")
# env.reset()
# 
# done = False
# 
# i = 0
# 
# while not done:
#     i += 1
#     
#     if state[1] > 0:
#         action = 2
#     else:
#         action = 0
#     state, reward, done, _ = env.step(action)
#     env.render()
#     print(f"Step {i}: State={state}, Reward = {reward}")
#     
# env.close()

In [2]:
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 10000
SHOW_EVERY = 20
buckets = 10

DISCRETE_GRID_SIZE = [10, 10]
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2

In [3]:
import gym
import numpy as np

def calc_discrete_state(state):
    discrete_state = (state - env.observation_space.low)/buckets
    return tuple(discrete_state.astype(np.int))  

def run_game(q_table, render, should_update):
    done = False
    discrete_state = calc_discrete_state(env.reset())
    success = False
    
    while not done:
        # Exploit or explore
        if np.random.random() > np.finfo(float).eps:
            # Exploit - use q-table to take current best action (and probably refine)
            action = np.argmax(q_table[discrete_state])
        else:
            # Explore - t
            action = np.random.randint(0, env.action_space.n)
            
        # Run simulation step
        new_state, reward, done, _ = env.step(action)
        
        # 
        new_state_disc = calc_discrete_state(new_state)

        # 
        if new_state[0] >= env.goal_position:
            success = True
          
        # Update q-table
        if should_update:
            max_future_q = np.max(q_table[new_state_disc])
            current_q = q_table[discrete_state + (action,)]
            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_state_disc
        
        if render:
            env.render()
            
    return success


In [ ]:
env = gym.make("MountainCar-v0")

epsilon = 1  
epsilon_change = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)
buckets = (env.observation_space.high - env.observation_space.low)/DISCRETE_GRID_SIZE
q_table = np.random.uniform(low=-3, high=0, size=(DISCRETE_GRID_SIZE + [env.action_space.n]))
success = False

In [ ]:
episode = 0
success_count = 0

while episode<EPISODES:
    episode+=1
    done = False

    if episode % SHOW_EVERY == 0:
        print(f"Current episode: {episode}, success: {success_count} ({float(success_count)/SHOW_EVERY})")
        success = run_game(q_table, True, False)
        success_count = 0
    else:
        success = run_game(q_table, False, True)
        
    if success:
        success_count += 1

    # Move epsilon towards its ending value, if it still needs to move
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
        epsilon -= epsilon_change

print(success)


Current episode: 20, success: 0 (0.0)
Current episode: 40, success: 0 (0.0)
Current episode: 60, success: 0 (0.0)
Current episode: 80, success: 0 (0.0)
Current episode: 100, success: 0 (0.0)
Current episode: 120, success: 0 (0.0)
Current episode: 140, success: 0 (0.0)
Current episode: 160, success: 0 (0.0)
Current episode: 180, success: 4 (0.2)
Current episode: 200, success: 0 (0.0)
Current episode: 220, success: 5 (0.25)
Current episode: 240, success: 0 (0.0)
Current episode: 260, success: 9 (0.45)
Current episode: 280, success: 1 (0.05)
Current episode: 300, success: 7 (0.35)
Current episode: 320, success: 0 (0.0)
Current episode: 340, success: 4 (0.2)
Current episode: 360, success: 6 (0.3)
Current episode: 380, success: 0 (0.0)
Current episode: 400, success: 3 (0.15)
Current episode: 420, success: 2 (0.1)
Current episode: 440, success: 0 (0.0)
Current episode: 460, success: 4 (0.2)
Current episode: 480, success: 11 (0.55)
Current episode: 500, success: 18 (0.9)
Current episode: 520,

In [ ]:
run_game(q_table, True, False)
